In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../../input/data/Casen2022.csv', low_memory=False)
data

,Unnamed: 0,id_vivienda,folio,id_persona,region,area,cod_upm,nse,estrato,hogar,...,men18c,may60c,tipohogar,tot_hog,ind_hacina,indsan,ten_viv,ten_viv_f,allega_ext,allega_int
0,1,1000901,100090101,1,16,2,10009,4,1630324,1,...,0,1,3,1,1,1,1,1,0,0
1,2,1000901,100090101,2,16,2,10009,4,1630324,1,...,0,1,3,1,1,1,1,1,0,0
2,3,1000901,100090101,3,16,2,10009,4,1630324,1,...,0,1,3,1,1,1,1,1,0,0
3,4,1000902,100090201,1,16,2,10009,4,1630324,1,...,1,1,5,1,1,2,1,1,0,1
4,5,1000902,100090201,2,16,2,10009,4,1630324,1,...,1,1,5,1,1,2,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202226,202227,4585804,458580401,2,9,1,45858,3,910113,1,...,1,0,3,1,1,1,1,1,0,0
202227,202228,4585804,458580401,3,9,1,45858,3,910113,1,...,1,0,3,1,1,1,1,1,0,0
202228,202229,4585807,458580701,1,9,1,45858,3,910113,1,...,1,0,3,1,1,1,1,1,0,0
202229,202230,4585807,458580701,2,9,1,45858,3,910113,1,...,1,0,3,1,1,1,1,1,0,0


In [3]:
datos_proc = data.copy()

In [4]:
# Exploración de variables I
print(data.shape, "\n\n")
print(data.columns, "\n\n")
print(data.dtypes, "\n\n")
print(data['sexo'].value_counts(), "\n\n")
print(data['region'].value_counts(), "\n\n")
print(data['edad'].value_counts(), "\n\n")
print(data['edad'].describe(), "\n\n")

(202231, 919) 


Index(['Unnamed: 0', 'id_vivienda', 'folio', 'id_persona', 'region', 'area',
       'cod_upm', 'nse', 'estrato', 'hogar',
       ...
       'men18c', 'may60c', 'tipohogar', 'tot_hog', 'ind_hacina', 'indsan',
       'ten_viv', 'ten_viv_f', 'allega_ext', 'allega_int'],
      dtype='object', length=919) 


Unnamed: 0     int64
id_vivienda    int64
folio          int64
id_persona     int64
region         int64
               ...  
indsan         int64
ten_viv        int64
ten_viv_f      int64
allega_ext     int64
allega_int     int64
Length: 919, dtype: object 


sexo
2    106575
1     95656
Name: count, dtype: int64 


region
13    38674
5     20552
8     19914
6     13864
7     13739
9     13577
10    10707
14    10603
2      9029
3      9011
16     8706
1      8691
4      8236
15     8152
12     5029
11     3747
Name: count, dtype: int64 


edad
30     3184
22     3101
60     3056
32     3030
50     2993
       ... 
103       5
107       2
105       2
104       2
120   

In [5]:
# Diccionario de Recodificación
reemplazos = {
    'o4': {2: 0},
    'o5': {3: 0},
    'o6': {2: 0},
    'o10': {-88: np.nan},
    'o12': {-88: np.nan, -99: np.nan},
    'o15': {-88: np.nan, -99: np.nan},
    'o18': {-88: np.nan, -99: np.nan},
    'o20': {-88: np.nan, -99: np.nan},
    'o21': {-88: np.nan, -99: np.nan},
    'y1': {-88: np.nan},
}

In [6]:
for col, repl in reemplazos.items():
    datos_proc[col] = datos_proc[col].replace(repl)

In [7]:
# Procesamiento
datos_proc = (
    datos_proc
    .assign(
        sexo_str=lambda x: x['sexo'].map({1: "Hombre", 2: "Mujer"}),

        sexo=lambda x: x['sexo'].map({1: 0, 2: 1}),
        
        edad_tr=lambda x: np.select([x['edad'].between(18, 39), 
                                         x['edad'].between(40, 64), x['edad'] >= 65],
                                        ["Jovenes", "Adultos", "Adultos mayores"],
                                        default=np.nan),
        
        sexo_edad=lambda x: np.select([(x['sexo_str'] == "Mujer") & (x['edad_tr'] == "Jovenes"),
                                       (x['sexo_str'] == "Mujer") & (x['edad_tr'] == "Adultos"),
                                       (x['sexo_str'] == "Mujer") & (x['edad_tr'] == "Adultos mayores"),
                                       (x['sexo_str'] == "Hombre") & (x['edad_tr'] == "Jovenes"),
                                       (x['sexo_str'] == "Hombre") & (x['edad_tr'] == "Adultos"),
                                       (x['sexo_str'] == "Hombre") & (x['edad_tr'] == "Adultos mayores")],
                                      ["Mujer joven", "Mujer adulta", "Mujer mayor", "Hombre joven", "Hombre adulto", "Hombre mayor"],
                                       default=np.nan),
        
        e6a_no_asiste=lambda x: np.select([x['e6a_no_asiste'].between(6, 7),
                                          x['e6a_no_asiste'].between(8, 11),
                                          x['e6a_no_asiste'] == 12,
                                          x['e6a_no_asiste'] == 13,
                                          x['e6a_no_asiste'].between(14, 15)],
                                         ["Basica", "Media", "Tecnico", "Universitaria", "Postgrado"],
                                          default="Otro"),

        region_str=lambda x: x['region'].map({1: "Región de Tarapacá",
                                         2: "Región de Antofagasta",
                                         3: "Región de Atacama",
                                         4: "Región de Coquimbo",
                                         5: "Región de Valparaíso",
                                         6: "Región de O'Higgins",
                                         7: "Región del Maule",
                                         8: "Región del Biobío",
                                         9: "Región de La Araucanía",
                                         10: "Región de Los Lagos",
                                         11: "Región de Aysén",
                                         12: "Región de Magallanes y Antártica",
                                         13: "Región Metropolitana",
                                         14: "Región de Los Ríos",
                                         15: "Región de Arica y Parinacota",
                                         16: "Región de Ñuble"}),
        
        educa_str=lambda x: np.select([x['e6a_no_asiste'] == "Otro",
                                  x['e6a_no_asiste'] == "Basica",
                                  x['e6a_no_asiste'] == "Media",
                                  (x['e6a_no_asiste'] == "Tecnico") & (x['e6c_completo'] == 1),
                                  (x['e6a_no_asiste'] == "Tecnico") & (x['e6c_completo'] == 2),
                                  (x['e6a_no_asiste'] == "Universitaria") & (x['e6c_completo'] == 1),
                                  (x['e6a_no_asiste'] == "Universitaria") & (x['e6c_completo'] == 2),
                                  (x['e6a_no_asiste'] == "Postgrado") & (x['e6c_completo'] == 1),
                                  (x['e6a_no_asiste'] == "Postgrado") & (x['e6c_completo'] == 2)],
                                 [np.nan, "Educación Basica", "Educación Media", 
                                  "Técnica completa", "Técnica incompleta", "Universitaria completa", 
                                  "Universitaria incompleta", "Postgrado", "Universitaria completa"]),
        
        educa=lambda x: np.select([x['e6a_no_asiste'] == "Otro",
                                  x['e6a_no_asiste'] == "Basica",
                                  x['e6a_no_asiste'] == "Media",
                                  (x['e6a_no_asiste'] == "Tecnico") & (x['e6c_completo'] == 1),
                                  (x['e6a_no_asiste'] == "Tecnico") & (x['e6c_completo'] == 2),
                                  (x['e6a_no_asiste'] == "Universitaria") & (x['e6c_completo'] == 1),
                                  (x['e6a_no_asiste'] == "Universitaria") & (x['e6c_completo'] == 2),
                                  (x['e6a_no_asiste'] == "Postgrado") & (x['e6c_completo'] == 1),
                                  (x['e6a_no_asiste'] == "Postgrado") & (x['e6c_completo'] == 2)],
                                 [np.nan, 1, 2, 4, 3, 6, 5, 7, 6]),
        
        temporada_trabajo_str=lambda x:x['o12'].map({
            1: 'Permanente',
            2: 'De temporada o estacional',
            3: 'Ocasional o eventual',
            4: 'A prueba',
            5: 'Por plazo o tiempo determinado'
        }),
        
        rol_laboral_str=lambda x:x['o15'].map({
            1: 'Patrón(a) o empleador(a)',
            2: 'Trabajador(a) por cuenta propia',
            3: 'Empleado(a) u obrero(a) del sector público (Gobierno Central o Municipal)',
            4: 'Empleado(a) u obrero(a) de empresas públicas',
            5: 'Empleado(a) u obrero(a) del sector privado',
            6: 'Servicio doméstico puertas adentro',
            7: 'Servicio doméstico puertas afuera',
            8: 'FF.AA. y del Orden',
            9: 'Familiar no remunerado',
        }),
        
        tipoContrato_str=lambda x:x['o18'].map({
            1: 'Plazo indefinido',
            2: 'Plazo fijo',
            3: 'Por obra o faena'
        }),
        
        jornadaLaboral_str=lambda x:x['o20'].map({
            1: 'Jornada completa',
            2: 'Jornada parcial',
            3: 'Jornada prolongada', # jornada extendida respecto al horario de acuerdo laboral
            4: 'Otra'
        }),
        
        tipoHorario_str=lambda x:x['o21'].map({
            1: 'Solo diurno',
            2: 'Solo nocturno',
            3: 'Rotativo o turnos'
        }))
    
    .rename(columns={'y1': 'ingreso'})
    .query('edad>=18'))

In [8]:
datos_proc = datos_proc[['sexo',
                         'sexo_str', 
                         'edad', 
                         'edad_tr', 
                         'sexo_edad', 
                         'region', 
                         'region_str', 
                         'educa', 
                         'educa_str', 
                         'ingreso',
                         'o1',
                         'o2',
                         'o3',
                         'o4',
                         'o5',
                         'o6',
                         'o7',
                         'o10',
                         'o12',
                         'temporada_trabajo_str',
                         'o15',
                         'rol_laboral_str',
                         'o18',
                         'tipoContrato_str',
                         'o20',
                         'jornadaLaboral_str',
                         'o21',
                         'tipoHorario_str',
                         ]]

datos_proc.rename(columns={
    'o4': 'TrabajaAlgunaVez',
    'o5': 'DisponibleTrabajar',
    'o6': 'Busca4semanas',
    'o7': 'RazonNoEmpleo',
    'o10': 'hrs_sem',
    'o12': 'temporada_trabajo',
    'o15': 'rol_laboral',
    'o18': 'tipoContrato',
    'o20': 'jornadaLaboral',
    'o21': 'tipoHorario',
}, inplace=True)

In [9]:
# Revisión de variables I
print(datos_proc.shape, "\n\n")
print(datos_proc.columns, "\n\n")
print(datos_proc.dtypes, "\n\n")
print(datos_proc['ingreso'].value_counts(), "\n\n")
print(datos_proc['ingreso'].describe(), "\n\n")
print(datos_proc['sexo_str'].value_counts(), "\n\n")
print(datos_proc['edad_tr'].value_counts(), "\n\n")
print(datos_proc['sexo_edad'].value_counts(), "\n\n")
print(datos_proc['educa_str'].value_counts(), "\n\n")
print(datos_proc['region_str'].value_counts(), "\n\n")

(158509, 28) 


Index(['sexo', 'sexo_str', 'edad', 'edad_tr', 'sexo_edad', 'region',
       'region_str', 'educa', 'educa_str', 'ingreso', 'o1', 'o2', 'o3',
       'TrabajaAlgunaVez', 'DisponibleTrabajar', 'Busca4semanas',
       'RazonNoEmpleo', 'hrs_sem', 'temporada_trabajo',
       'temporada_trabajo_str', 'rol_laboral', 'rol_laboral_str',
       'tipoContrato', 'tipoContrato_str', 'jornadaLaboral',
       'jornadaLaboral_str', 'tipoHorario', 'tipoHorario_str'],
      dtype='object') 


sexo                       int64
sexo_str                  object
edad                       int64
edad_tr                   object
sexo_edad                 object
region                     int64
region_str                object
educa                    float64
educa_str                 object
ingreso                  float64
o1                       float64
o2                       float64
o3                       float64
TrabajaAlgunaVez         float64
DisponibleTrabajar       float64
Busca4sema

In [10]:
# Revisión de variables II
print(datos_proc['hrs_sem'].describe(), '\n\n')
print(datos_proc['temporada_trabajo_str'].value_counts().sort_index(), '\n\n')
print(datos_proc['rol_laboral_str'].value_counts().sort_index(), '\n\n')
print(datos_proc['tipoContrato_str'].value_counts().sort_index(), '\n\n')
print(datos_proc['jornadaLaboral_str'].value_counts().sort_index(), '\n\n')
print(datos_proc['tipoHorario_str'].value_counts().sort_index(), '\n\n')

count    84340.000000
mean        40.877863
std         13.303460
min          0.000000
25%         40.000000
50%         45.000000
75%         45.000000
max        168.000000
Name: hrs_sem, dtype: float64 


temporada_trabajo_str
A prueba                            510
De temporada o estacional          8342
Ocasional o eventual               8463
Permanente                        64049
Por plazo o tiempo determinado     3948
Name: count, dtype: int64 


rol_laboral_str
Empleado(a) u obrero(a) de empresas públicas                                  3365
Empleado(a) u obrero(a) del sector privado                                   46620
Empleado(a) u obrero(a) del sector público (Gobierno Central o Municipal)     8496
FF.AA. y del Orden                                                             619
Familiar no remunerado                                                         362
Patrón(a) o empleador(a)                                                      3231
Servicio doméstico puertas

In [11]:
# Guardar datos
datos_proc.to_csv("../../output/data/Casen2022_proc.csv", index=False)